# Human Body Dimension Estimation (HBDE) from occluded images

### Authors: M. Beiwinkler, M. Krimpelstätter, I. Viertola and T. Wulz

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import glob
import random
from skimage.io import imread
from PIL import Image, ImageDraw

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.nn import Linear, ReLU, MSELoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, Flatten
from torch.optim import Adam, SGD
from torchvision.transforms import ToTensor

%matplotlib inline

In [2]:
# Paths
DATASET_PATH = os.path.join(os.getcwd(), "dataset")
TRAIN_TEST_PATH = os.path.join(DATASET_PATH, os.path.join("train_test", "train_test_split.json"))
IMS_PATH = os.path.join(DATASET_PATH, os.path.join("synthetic_images", "200x200"))
ANNOS_PATH = os.path.join(DATASET_PATH, "annotations")
FULL_DATA_NP_ARRAY_PATH = os.path.join(os.getcwd(), "im_data.npy")
FULL_DATA_WITH_OCCLUSIONS_NP_ARRAY_PATH = os.path.join(os.getcwd(), "im_data_with_occlusions.npy")
CHOSEN_ANNOS_NP_ARRAY_PATH = os.path.join(os.getcwd(), "im_annos.npy")

if not os.path.exists(IMS_PATH):
    print("Dataset does not exist {}".format(DATASET_PATH))
    exit()

## Data preparation

In [6]:
 class ImageDataset(Dataset):
    def __init__(self, with_occlusions=False, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        if with_occlusions and os.path.exists(FULL_DATA_WITH_OCCLUSIONS_NP_ARRAY_PATH):
            # Load occluded images from file
            self.im_data = np.load(FULL_DATA_WITH_OCCLUSIONS_NP_ARRAY_PATH)
            if not self.im_data.shape == (12000, 200, 200):
                print("Problems with occluded image data.")
                exit()
        else:
            # Load data from file or from the dataset
            if os.path.exists(FULL_DATA_NP_ARRAY_PATH):
                self.im_data = np.load(FULL_DATA_NP_ARRAY_PATH)
                self.im_annos = np.load(CHOSEN_ANNOS_NP_ARRAY_PATH)
            else:
                im_data = []
                im_annos = []
                for anno_fn in sorted(glob.glob(os.path.join(ANNOS_PATH, os.path.join("*", "*.json")))):
                    f = open(anno_fn, "r")
                    annotations = json.load(f)
                    im_annos.append([annotations['human_dimensions']['height'], annotations['human_dimensions']['shoulder_width'],
                                     annotations['human_dimensions']['left_arm_length'], annotations['human_dimensions']['right_arm_length'],
                                     annotations['human_dimensions']['pelvis_circumference'], annotations['human_dimensions']['chest_circumference']])
                for img_fn in sorted(glob.glob(os.path.join(IMS_PATH, os.path.join("**", "*.png")), recursive=True)):
                    img = imread(img_fn)
                    img = img.astype('float32')
                    img /= 255.0
                    im_data.append(img)

                self.im_data = np.array(im_data)
                self.im_annos = np.array(im_annos)
                with open(FULL_DATA_NP_ARRAY_PATH, "wb+") as f:
                    np.save(f, self.im_data)
                    f.close()
                with open(CHOSEN_ANNOS_NP_ARRAY_PATH, "wb+") as f:
                    np.save(f, self.im_annos)
                    f.close()

            if not self.im_data.shape == (12000, 200, 200):
                print("Problems with image data.")
                exit()
            if not self.im_annos.shape == (6000, 6):
                print("Problems with image data.")
                exit()
                    
            if with_occlusions:
                # Generate occluded images
                # Seed pseudo-random number generator for reproducibility
                random.seed(a="HBDE from occluded images", version=2)

                # Add rectangles to images
                im_data_occl = []
                for img_np in self.im_data:
                    # Determine position, length and width of occluding rectangle
                    upper_left_x = random.randrange(0, 200)
                    upper_left_y = random.randrange(0, 200)
                    lower_right_x = upper_left_x + random.randrange(0, 50)
                    lower_right_y = upper_left_y + random.randrange(0, 50)

                    # Convert from NumPy array to instance of Pillow Image class
                    img = Image.fromarray(img_np)

                    # Draw rectangles
                    draw = ImageDraw.Draw(img)
                    draw.rectangle((upper_left_x, upper_left_y, lower_right_x, lower_right_y), fill="black")

                    # Add manipulated image to array of occluded images
                    im_data_occl.append(np.asarray(img))
        
                self.im_data = np.array(im_data_occl)
                with open(FULL_DATA_WITH_OCCLUSIONS_NP_ARRAY_PATH, "wb+") as f:
                    np.save(f, self.im_data)
                    f.close()
                if not self.im_data.shape == (12000, 200, 200):
                    print("Problems with occluded image data.")
                    exit()
        # Review data
        #plt.figure(figsize=(15, 15))
        #plt.subplot(221), plt.imshow(self.im_data[0], cmap='gray'), plt.title("Female, pose 0")
        #plt.subplot(222), plt.imshow(self.im_data[3001], cmap='gray'), plt.title("Male, pose 0")
        #plt.subplot(223), plt.imshow(self.im_data[6001], cmap='gray'), plt.title("Female, pose 1")
        #plt.subplot(224), plt.imshow(self.im_data[9001], cmap='gray'), plt.title("Male, pose 1")

    def __len__(self):
        return len(self.im_data)

    def __getitem__(self, idx):
        image = self.im_data[idx]
        #print(f"{idx}, {len(self.im_annos)}, {idx % len(self.im_annos)}")
        annotations = np.float32(self.im_annos[idx % len(self.im_annos)])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            annotations = self.target_transform(annotations)
        #print(f"{image.dtype} {annotations.dtype}")
        return image, annotations

## Create the CNN
Selected features (6): 
- height
- shoulder width
- left arm length
- right arm length
- pelvis circumference
- chest circumference


In [7]:
feature_amnt = 6

class CNN(Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layers = Sequential(
            Conv2d(in_channels=1, out_channels=feature_amnt, kernel_size=5),
            ReLU(inplace=True),
            BatchNorm2d(num_features=feature_amnt),
            MaxPool2d(kernel_size=5, stride=2),
            # Second convolutional layer
            Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            ReLU(inplace=True),  # Are these irrelevant?
            BatchNorm2d(num_features=16),  # Are these irrelevant?
            MaxPool2d(kernel_size=5, stride=2),
            Flatten(),
            Linear(in_features=30976, out_features=30976),  # Drop the out feature amount here?
            ReLU(inplace=True),
            Linear(in_features=30976, out_features=feature_amnt)
        )
    
    def forward(self, x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)
        return x

model = CNN()
print(model)

CNN(
  (layers): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU(inplace=True)
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=30976, out_features=30976, bias=True)
    (10): ReLU(inplace=True)
    (11): Linear(in_features=30976, out_features=6, bias=True)
  )
)


## Train the model

In [ ]:
training_data = ImageDataset(transform=ToTensor())
test_data = ImageDataset(with_occlusions=True, transform=ToTensor(), target_transform=ToTensor())

train_dataloader = DataLoader(training_data, batch_size=100)
test_dataloader = DataLoader(test_data, batch_size=100)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# optimizer
learning_rate = 0.01
momentum = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
# the loss function
criterion = MSELoss()


epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, criterion, optimizer)
    test_loop(test_dataloader, model, criterion)
print("Done!")


Epoch 1
-------------------------------
